In [15]:
import requests
import nest
import pprint

In [2]:
response = requests.get('http://localhost:5005')
data = response.json()
print(data)

{'nest': 'master@01f6e5a76', 'nestml': '5.0.0-rc1'}


In [10]:
def script(name):
    return f"""neuron {name}:

  state:
    r integer = 0                 # counts number of tick during the refractory period
    V_abs mV = 0 mV
  end

  equations:
    kernel I_kernel_inh = (e / tau_syn_inh) * t * exp(-t / tau_syn_inh)
    kernel I_kernel_exc = (e / tau_syn_exc) * t * exp(-t / tau_syn_exc)
    recordable inline V_m mV = V_abs + E_L # Membrane potential.
    inline I pA = convolve(I_kernel_inh, inh_spikes) + convolve(I_kernel_exc, exc_spikes) + I_e + I_stim
    V_abs' = -V_abs/tau_m + I/C_m
  end

  parameters:
    C_m     pF = 250 pF       # Capacitance of the membrane
    tau_m   ms = 10 ms        # Membrane time constant
    tau_syn_inh ms = 2 ms     # Time constant of synaptic current
    tau_syn_exc ms = 2 ms     # Time constant of synaptic current
    t_ref   ms = 2 ms         # Duration of refractory period
    E_L     mV = -70 mV       # Resting potential
    V_reset mV = -70 mV - E_L # Reset potential of the membrane
    V_th   mV = -55 mV - E_L  # Spike threshold
    I_e pA = 0 pA
  end

  internals:
    RefractoryCounts integer = steps(t_ref) # refractory time in steps
  end

  input:
    exc_spikes pA <- excitatory spike
    inh_spikes pA <- inhibitory spike
    I_stim pA <- continuous
  end

  output: spike

  update:
    if r == 0: # neuron not refractory
      integrate_odes()
    else: # neuron is absolute refractory
      r = r - 1
    end

    if V_abs >= V_th: # threshold crossing
      r = RefractoryCounts
      V_abs = V_reset
      emit_spike()
    end

  end

end
"""

In [21]:
models = ['my_neuron1', 'my_neuron2', 'my_neuron3'][:2]
data = {'models': [{'name': model, 'script': script(model)} for model in models]}
pprint.pprint(data)

{'models': [{'name': 'my_neuron1',
             'script': 'neuron my_neuron1:\n'
                       '\n'
                       '  state:\n'
                       '    r integer = 0                 # counts number of '
                       'tick during the refractory period\n'
                       '    V_abs mV = 0 mV\n'
                       '  end\n'
                       '\n'
                       '  equations:\n'
                       '    kernel I_kernel_inh = (e / tau_syn_inh) * t * '
                       'exp(-t / tau_syn_inh)\n'
                       '    kernel I_kernel_exc = (e / tau_syn_exc) * t * '
                       'exp(-t / tau_syn_exc)\n'
                       '    recordable inline V_m mV = V_abs + E_L # Membrane '
                       'potential.\n'
                       '    inline I pA = convolve(I_kernel_inh, inh_spikes) + '
                       'convolve(I_kernel_exc, exc_spikes) + I_e + I_stim\n'
                       "    V_abs' = -V_a

In [22]:
response = requests.post('http://localhost:5005/install', json=data)

try:
    pprint.pprint(response.json())
except:
    print(response.text)

{'status': {'BUILT': ['nestmlmodule', 'my_neuron1', 'my_neuron2'],
            'INITIALIZED': ['my_neuron1', 'my_neuron2'],
            'INSTALLED': [],
            'WRITTEN': ['my_neuron2', 'my_neuron1']}}


In [1]:
import nest
nest.ResetKernel()
try:
    nest.Install('nestmlmodule')
except:
    pass


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: UNKNOWN
 Built: Mar 30 2022 15:28:56

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.


Mar 30 17:08:51 Install [Info]: 
    loaded module nestmlmodule


In [2]:
models = nest.node_models
my_models = [model for model in models if 'my' in model]
print(my_models)

['my_neuron3']


In [ ]:
data = {'models': [{'name': model_name, 'script': script}]}
data